<a href="https://colab.research.google.com/github/mortymh/moryton/blob/master/California%20House%20Price%20Prediction%20Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 01_Data_exploration
'''
در این بخش داده‌ها از گوگل درایو بارگذاری شده و به صورت اولیه بررسی می‌شوند:
- نمایش چند سطر اول داده‌ها
- بررسی نوع داده‌ها و وجود مقادیر گمشده
- آمار توصیفی داده‌ها
'''

from google.colab import drive
drive.mount('/content/drive')

import pandas as pd

# مسیر فایل CSV — لطفاً این مسیر را با مسیر واقعی فایل‌تان تغییر دهید
file_path = "/content/drive/MyDrive/datasets/housing.csv"

# بارگذاری داده‌ها
df = pd.read_csv(file_path)

# نمایش اولین 5 ردیف از داده‌ها
print(df.head())

# بررسی اطلاعات کلی دیتافریم (نوع داده، تعداد مقادیر غیر خالی و ...)
print(df.info())

# نمایش آمار توصیفی از داده‌ها (میانگین، انحراف معیار، کمینه، بیشینه و ...)
print(df.describe())

# بررسی تعداد مقادیر گمشده در هر ستون
print(df.isnull().sum())


Mounted at /content/drive
   longitude  latitude  housing_median_age  total_rooms  total_bedrooms  \
0    -122.23     37.88                41.0        880.0           129.0   
1    -122.22     37.86                21.0       7099.0          1106.0   
2    -122.24     37.85                52.0       1467.0           190.0   
3    -122.25     37.85                52.0       1274.0           235.0   
4    -122.25     37.85                52.0       1627.0           280.0   

   population  households  median_income  median_house_value ocean_proximity  
0       322.0       126.0         8.3252            452600.0        NEAR BAY  
1      2401.0      1138.0         8.3014            358500.0        NEAR BAY  
2       496.0       177.0         7.2574            352100.0        NEAR BAY  
3       558.0       219.0         5.6431            341300.0        NEAR BAY  
4       565.0       259.0         3.8462            342200.0        NEAR BAY  
<class 'pandas.core.frame.DataFrame'>
RangeIndex:

In [2]:
# 02_preprocessing_and_split
'''
این سلول وظیفه پیش‌پردازش داده‌ها و آماده‌سازی برای آموزش مدل را دارد:

1. پر کردن مقادیر گمشده در ستون 'total_bedrooms' با میانگین همان ستون.
2. تبدیل ستون متنی 'ocean_proximity' به داده عددی با استفاده از LabelEncoder.
3. جدا کردن ویژگی‌ها (features) از ستون هدف (target) یعنی 'median_house_value'.
4. نرمال‌سازی (استانداردسازی) داده‌ها با StandardScaler.
5. تبدیل آرایه‌های نرمال‌شده به DataFrame و افزودن دوباره ستون هدف.
6. تقسیم داده‌ها به داده‌های آموزش و تست با نسبت ۸۰ به ۲۰.
'''

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import pandas as pd

# پر کردن مقادیر گمشده با میانگین همان ستون
df['total_bedrooms'] = df['total_bedrooms'].fillna(df['total_bedrooms'].mean())

# ساخت LabelEncoder برای تبدیل داده متنی به عددی
encoder = LabelEncoder()
df['ocean_proximity'] = encoder.fit_transform(df['ocean_proximity'])

# جدا کردن ستون هدف (target) از بقیه ویژگی‌ها (features)
features = df.drop('median_house_value', axis=1)

# نرمال‌سازی ویژگی‌ها برای بهبود عملکرد مدل
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

# تبدیل آرایه نرمال‌شده به DataFrame با نام ستون‌های قبلی
df_scaled = pd.DataFrame(scaled_features, columns=features.columns)

# افزودن ستون هدف به DataFrame جدید
df_scaled['median_house_value'] = df['median_house_value']

# تعیین ویژگی‌ها و هدف برای مدل
X = df_scaled.drop('median_house_value', axis=1)
y = df_scaled['median_house_value']

# تقسیم داده‌ها به آموزش و تست (۸۰٪ آموزش و ۲۰٪ تست)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)


In [3]:
# 03_linear_regression
'''
در این سلول مدل رگرسیون خطی ساخته و آموزش داده می‌شود و سپس عملکرد آن روی داده‌های تست ارزیابی می‌شود:

1. ساخت مدل رگرسیون خطی با LinearRegression.
2. آموزش مدل با داده‌های آموزش (X_train و y_train).
3. پیش‌بینی مقدار هدف (قیمت خانه) روی داده‌های تست.
4. محاسبه معیارهای ارزیابی:
   - میانگین مربع خطا (MSE): خطای متوسط توان دوم پیش‌بینی‌ها.
   - میانگین قدر مطلق خطا (MAE): میانگین اختلاف مطلق پیش‌بینی‌ها.
   - ضریب تعیین (R2): میزان توضیح‌داده‌شده توسط مدل (حداکثر 1، هرچه نزدیک‌تر بهتر).
5. چاپ نتایج برای تحلیل عملکرد مدل.
'''

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# ساخت مدل رگرسیون خطی
model = LinearRegression()

# آموزش مدل روی داده‌های آموزش
model.fit(X_train, y_train)

# پیش‌بینی روی داده‌های تست
y_pred = model.predict(X_test)

# محاسبه خطاهای مدل
mse = mean_squared_error(y_test, y_pred)      # میانگین مربع خطا
mae = mean_absolute_error(y_test, y_pred)     # میانگین قدر مطلق خطا
r2 = r2_score(y_test, y_pred)                  # ضریب تعیین

# نمایش نتایج
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"R-squared (R2): {r2:.2f}")


Mean Squared Error (MSE): 5055025116.17
Mean Absolute Error (MAE): 51846.88
R-squared (R2): 0.61


In [4]:
#04_model_improvement

'''
در این سلول مدل‌های بهبود یافته رگرسیون خطی یعنی Ridge و Lasso بررسی می‌شوند.
هدف این است که با افزودن منظم‌سازی (Regularization) مدل را قوی‌تر و پایدارتر کنیم:

1. Ridge Regression:
   - با پارامتر alpha=1.0 ساخته شده که میزان منظم‌سازی را تعیین می‌کند.
   - مدل روی داده‌های آموزش آموزش داده می‌شود.
   - پیش‌بینی روی داده‌های تست انجام می‌شود.
   - معیارهای MSE و R2 محاسبه و نمایش داده می‌شوند.

2. Lasso Regression:
   - با پارامتر alpha=0.1 ساخته شده (منظم‌سازی ملایم‌تر).
   - آموزش و پیش‌بینی مانند Ridge انجام می‌شود.
   - معیارهای MSE و R2 محاسبه و نمایش داده می‌شوند.

منظم‌سازی به کاهش Overfitting و بهبود تعمیم‌پذیری مدل کمک می‌کند.
'''

from sklearn.linear_model import Ridge, Lasso
from sklearn.metrics import mean_squared_error, r2_score

# مدل Ridge Regression
ridge = Ridge(alpha=1.0)  # alpha میزان قدرت منظم‌سازی را کنترل می‌کند
ridge.fit(X_train, y_train)  # آموزش مدل روی داده‌های آموزش

y_pred_ridge = ridge.predict(X_test)  # پیش‌بینی روی داده‌های تست

# ارزیابی مدل Ridge
mse_ridge = mean_squared_error(y_test, y_pred_ridge)
r2_ridge = r2_score(y_test, y_pred_ridge)

print("Ridge Regression:")
print(f"Mean Squared Error: {mse_ridge:.2f}")
print(f"R2 Score: {r2_ridge:.2f}")

# مدل Lasso Regression
lasso = Lasso(alpha=0.1)  # alpha کمتر، منظم‌سازی ملایم‌تر
lasso.fit(X_train, y_train)  # آموزش مدل

y_pred_lasso = lasso.predict(X_test)  # پیش‌بینی روی داده‌های تست

# ارزیابی مدل Lasso
mse_lasso = mean_squared_error(y_test, y_pred_lasso)
r2_lasso = r2_score(y_test, y_pred_lasso)

print("\nLasso Regression:")
print(f"Mean Squared Error: {mse_lasso:.2f}")
print(f"R2 Score: {r2_lasso:.2f}")


Ridge Regression:
Mean Squared Error: 5054505291.71
R2 Score: 0.61

Lasso Regression:
Mean Squared Error: 5055015786.77
R2 Score: 0.61
